In [91]:
from tqdm.notebook import tqdm
import MeCab
import numpy as np
import re
import pickle
import matplotlib.pyplot as plt
import string
import warnings
import gensim
import pyLDAvis
import pyLDAvis.gensim

warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
def pos(text):
    p = re.compile(".+\t[A-Z]+")
    return [tuple(p.match(line).group().split("\t")) for line in MeCab.Tagger().parse(text).splitlines()[:-1]]

def morphs(text):
    p = re.compile(".+\t[A-Z]+")
    return [p.match(line).group().split("\t")[0] for line in MeCab.Tagger().parse(text).splitlines()[:-1]]

def nouns(text):
    p = re.compile(".+\t[A-Z]+")
    temp = [tuple(p.match(line).group().split("\t")) for line in MeCab.Tagger().parse(text).splitlines()[:-1]]
    nouns=[]
    for word in temp:
        if word[1] in ["NNG", "NNP", "NNB", "NNBC", "NP", "NR"]:
            nouns.append(word[0])
    return nouns

def cln(docs):
    # 한국어를 제외한 글자를 제거하는 함수.
    for doc in docs:
        doc = re.sub("[^ㄱ-ㅣ가-힣 ]", "", doc)
    return docs

def def_sw(path):
    sw = set()
    for i in string.punctuation:
        sw.add(i)
    with open(path, encoding="utf-8") as f:
        for word in f:
            sw.add(word.split("\n")[0])
    return sw

def tkn(doc, tokenizer):
    token_doc = []
    if tokenizer == "noun":
        for n in tqdm(range(len(doc)), desc="Preprocessing"):
            token_text = nouns(doc[n])
            token_text = [word for word in token_text if word not in sw and len(word) > 1]
            token_doc.append(token_text)
            
    elif tokenized == "morph":
        for n in tqdm(range(len(doc)), desc="Preprocessing"):
            token_text = morphs(doc[n])
            token_text = [word for word in token_text if word not in sw and len(word) > 1]
            token_doc.append(token_text)

    elif tokenizer == "word":
        for n in tqdm(range(len(doc)), desc="Preprocessing"):
            token_text = doc[n].split()
            token_text = [word for word in token_text if word not in sw and len(word) > 1]
            token_doc.append(token_text)

    return token_doc

In [6]:
with open("naver_news.pk", "rb") as f:
    docs = pickle.load(f)

In [10]:
sw = def_sw("C:/Users/5CG7092POZ/stopwords-ko.txt")

docs_cln = cln(docs)
docs_tkn = tkn(docs_cln, tokenizer="noun") 

In [66]:
id2word = gensim.corpora.Dictionary(docs_tkn)

dtm = [id2word.doc2bow(doc) for doc in docs_tkn]

model = gensim.models.ldamodel.LdaModel(dtm, num_topics=4, id2word=id2word)

In [67]:
model.show_topic(3, 10)

[('사업', 0.013356806),
 ('개발', 0.013189332),
 ('경제', 0.009387829),
 ('규모', 0.008275956),
 ('속초', 0.007960775),
 ('예정', 0.007901422),
 ('뉴스', 0.007869918),
 ('부동산', 0.0069992193),
 ('현대산업개발', 0.006252979),
 ('역세', 0.005703076)]

In [72]:
n_topics = 3
n_topic_words = 30

model = gensim.models.ldamodel.LdaModel(dtm, num_topics=n_topics, id2word=id2word, alpha="auto", eta="auto")

In [92]:
pyLDAvis.enable_notebook()

pyldavis = pyLDAvis.gensim.prepare(model, dtm, id2word)

In [93]:
pyldavis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.019577  0.015039       1        1  50.521244
2      0.038427  0.000191       2        1  29.242796
0     -0.018850 -0.015230       3        1  20.235964, topic_info=       Term       Freq      Total Category  logprob  loglift
33       속초  29.000000  29.000000  Default  30.0000  30.0000
827      제기  23.000000  23.000000  Default  29.0000  29.0000
70   현대산업개발  78.000000  78.000000  Default  28.0000  28.0000
476      구역  30.000000  30.000000  Default  27.0000  27.0000
525      정비  19.000000  19.000000  Default  26.0000  26.0000
..      ...        ...        ...      ...      ...      ...
18       뉴스   4.884731  33.951645   Topic3  -5.3187  -0.3411
0        가구   3.508842  14.520542   Topic3  -5.6495   0.1774
303      시장   3.673126  17.947798   Topic3  -5.6037   0.0113
506      수주   3.885365  31.322060   Topic3  -5.5476  -0.4894
25       무단   3.834226  29.101147   Topic3  -5.5608  -0.4291

[229 rows x 6 columns], token_table=      Topic      Freq Term
term                      
0         1  0.550944   가구
0         2  0.275472   가구
0         3  0.275472   가구
552       1  0.338387   가비
552       3  0.676774   가비
...     ...       ...  ...
849       1  0.293745   회생
849       2  0.587490   회생
849       3  0.146873   회생
604       1  0.444386   후속
604       3  0.444386   후속

[408 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 1])